<a href="https://colab.research.google.com/github/Bhupeshgurav/Sentiment-Analysis/blob/main/sentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install and Import Dependencies**

In [ ]:
!pip3 install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.5 MB/s eta 0:13:29tcmalloc: large alloc 1147494400 bytes == 0x556aac38e000 @  0x7fa3deacf615 0x556a73b1e4cc 0x556a73bfe47a 0x556a73b212ed 0x556a73c12e1d 0x556a73b94e99 0x556a73b8f9ee 0x556a73b22bda 0x556a73b94d00 0x556a73b8f9ee 0x556a73b22bda 0x556a73b91737 0x556a73c13c66 0x556a73b90daf 0x556a73c13c66 0x556a73b90daf 0x556a73c13c66 0x556a73b90daf 0x556a73b23039 0x556a73b66409 0x556a73b21c52 0x556a73b94c25 0x556a73b8f9ee 0x556a73b22bda 0x556a73b91737 0x556a73b8f9ee 0x556a73b22bda 0x556a73b90915 0x556a73b22afa 0x556a73b90c0d 0x556a73b8f9ee
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:15tcmalloc: large alloc 1434370048 bytes == 0x556af09e4000 @  0x7fa3deacf615 0x556a73b1e4cc 0x556a73bfe47a 0x556a73b212ed 0x556a73c12e1d 0x556a73b94e99 0x556a73b8f9ee 0x556a73b22bda 0x556a73b94d00 0x556a73b8f9ee 0x556a73b22bda 0x556a73b91737 0x556a73c13c66 0x556a73b90d

In [ ]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.8 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 55.9 MB/s 
     |████████████████████████████████| 3.3 MB 28.4 MB/s 
     |████████████████████████████████| 636 kB 63.0 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# **2 Instantiate Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

Encode and Calculate Sentiment


In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [ ]:
int(torch.argmax(result.logits))+1 
                                    #extracts our sentiment score
                                    # higher the number the better the sentiment

4

# **Collect Reviews**

In [ ]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex}) # text wrapped in p tag and has a class which matches our regex
reviews = [result.text for result in results] #removed the tags 

In [ ]:
r.text # to get the text out of that webpage, then pass this text to beautifulsoap, then extracting the specific components of the webstie

In [ ]:
results[0].text # gave us the text which we wanted

'Social brew cafe is the best cafe for brunch really enjoyed it with my friends.Supply top quality corvid 19 vaccination cards .Contact now for more information.best quality. \xa0contact us at :corvidvacccine@gmail.com'

In [ ]:
reviews[0]

'Social brew cafe is the best cafe for brunch really enjoyed it with my friends.Supply top quality corvid 19 vaccination cards .Contact now for more information.best quality. \xa0contact us at :corvidvacccine@gmail.com'

# **Load Reviews into DataFrame and Score**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.array(reviews), columns = ['review']) # got our reviews in a data frame

In [ ]:
df.head() # to see our first 5 reviews

,review
0,Social brew cafe is the best cafe for brunch r...
1,I came to Social brew cafe for brunch while ex...
2,I went here a little while ago- a beautiful mo...
3,Ricotta hot cakes! These were so yummy. I ate ...
4,Good coffee and toasts. Straight up and down -...


In [ ]:
df.tail()  # to see our last five reviews

,review
6,We came for brunch twice in our week-long visi...
7,Great coffee and vibe. That's all you need. C...
8,Great coffee and vibe. That's all you need. C...
9,This cafe is one of the most popular cafes whe...
10,This is my fave brunch café in and around Sydn...


In [ ]:
df['review'].iloc[0]

'Social brew cafe is the best cafe for brunch really enjoyed it with my friends.Supply top quality corvid 19 vaccination cards .Contact now for more information.best quality. \xa0contact us at :corvidvacccine@gmail.com'

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df['review']

0     Social brew cafe is the best cafe for brunch r...
1     I came to Social brew cafe for brunch while ex...
2     I went here a little while ago- a beautiful mo...
3     Ricotta hot cakes! These were so yummy. I ate ...
4     Good coffee and toasts. Straight up and down -...
5     Ron & Jo are on the go down under and Wow!  We...
6     We came for brunch twice in our week-long visi...
7     Great coffee and vibe. That's all  you need. C...
8     Great coffee and vibe. That's all  you need. C...
9     This cafe is one of the most popular cafes whe...
10    This is my fave brunch café in and around Sydn...
Name: review, dtype: object

In [ ]:
df # We got our sentiment score fully calculated "YEPPY" 5 is best 1 is worst

,review,sentiment
0,Social brew cafe is the best cafe for brunch r...,5
1,I came to Social brew cafe for brunch while ex...,5
2,I went here a little while ago- a beautiful mo...,2
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,Good coffee and toasts. Straight up and down -...,5
5,Ron & Jo are on the go down under and Wow! We...,5
6,We came for brunch twice in our week-long visi...,4
7,Great coffee and vibe. That's all you need. C...,5
8,Great coffee and vibe. That's all you need. C...,4
9,This cafe is one of the most popular cafes whe...,5


In [ ]:
 df['review'].iloc[3] # for checking other

"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"